In [19]:
import pandas as pd

In [20]:
df = pd.read_csv("spotify_millsongdata.csv")

In [21]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [22]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [23]:
df.shape

(57650, 4)

In [24]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [25]:
df =df.sample(10000).drop('link', axis=1).reset_index(drop=True)

In [26]:
df.head(10)

,artist,song,text
0,Linkin Park,A Light That Never Comes,"Nah, you don't know me \r\nLightning above an..."
1,Vertical Horizon,Heart In Hand,I know it's been a long road \r\nTo get these...
2,Travis,The Distance,Do you remember? \r\nDo you remember? \r\nDo...
3,W.A.S.P.,Mississippi Queen,Mississippi Queen - you know what I mean \r\n...
4,Black Sabbath,Hot Line,Take me to the river baby drink my wine \r\nW...
5,Kanye West,Swagga Like Us,[Kanye West] \r\nMr. West is in the buildin ...
6,Air Supply,Empty Pages,When you're out in front and your life is unsu...
7,James Taylor,Sunshine Sunshine,"Sounds of laughter, here comes sunshine, smili..."
8,Gary Numan,Bombers,Look up I hear the scream of sirens on the wal...
9,Leonard Cohen,You Know Who I Am,"I cannot follow you, my love, \r\nYou cannot ..."


In [27]:
df['text'][0]

'Nah, you don\'t know me  \r\nLightning above and a fire below me  \r\nYou cannot catch me, cannot hold me  \r\nYou cannot stop, much less control me  \r\nWhen it rains, it pours, when the floodgates open brace your shores  \r\nThat pressure don\'t care when it breaks your doors  \r\nSay it\'s all you can take, better take some more  \r\n  \r\n(Oh)  \r\n\'Cause I know what it\'s like to test fate  \r\nHad my shoulders pressed with that weight  \r\nStood up strong in spite of that hate  \r\n  \r\n(Oh)  \r\nNight gets darkest right before dawn  \r\nWhat don\'t kill you makes you more strong  \r\nAnd I been waiting for it so long  \r\n  \r\nThe nights go on  \r\nWaiting for a light that never comes  \r\nI chase the sun  \r\nWaiting for a light that never comes  \r\n  \r\nOh  \r\nOh  \r\nOh  \r\nWaiting for a light that never comes  \r\n  \r\nWhen I was young, they told me, they said  \r\nMake your bed, you lie in that bed  \r\nA king can only reign \'til instead  \r\nThere comes that day 

In [28]:
# df = df.sample(5000)

In [29]:
df.shape

(10000, 3)

Text Cleaning/ Text Preprocessing

In [30]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [33]:
!pip install nltk
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/d3/10/6f2d5f8635d7714ad97ce6ade7a643358c4f3e45cde4ed12b7150734a8f3/regex-2023.10.3-cp312-cp312-win_amd64.whl.metadata
  Using cached regex-2023.10.3-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached regex-2023.10.3-cp312-cp312-win_amd64.whl (268 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


In [34]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [37]:
similarity[0]

array([1.        , 0.09085072, 0.02576843, ..., 0.03066256, 0.00649212,
       0.03532254])

In [38]:
df[df['song'] == 'Crying Over You']

,artist,song,text
5107,ABBA,Crying Over You,i 'm waitin ' for you babi i 'm sit all alon i...


In [39]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [40]:
recommendation('Crying Over You')

["Cryin' Time Again",
 'Crying Time',
 'If I Could',
 'If I Cried Every Time You Hurt Me',
 'Change My Way Of Living',
 'Psalm 8',
 'Back Into Blue',
 'Morning Dew',
 'How Majestic Is Thy Name',
 'Vagabond Of The Western World',
 'Baby Blues',
 "Jamie's Cryin'",
 'Dear Lord',
 'Blue Eyes',
 'You Move Me',
 'Closer Every Day',
 'Lord, In Your Love',
 'Color Of The Blues',
 "I'm In A Blue, Blue Mood",
 'Blue Christmas']

In [41]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))